In [14]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import argparse
import matplotlib.pyplot as plt 
from pprint import pprint
import pandas as pd 

In [15]:
required_vehicle_rate = 0.41
network_file = os.path.abspath("data/data/4x4grid_similar_to_resco_train_new_files")
path_to_save_all_route_data = os.path.abspath("data/4x4grid_similar_to_resco_train_new_files")

In [16]:
# os.environ['SUMO_HOME'] = '/opt/homebrew/bin/sumo' 
os.environ['SUMO_HOME'] = '/opt/homebrew/opt/sumo/bin' 

 # Adjust this path to your SUMO installation
import sys
tools = '/opt/homebrew/opt/sumo/share/sumo/tools'
sys.path.append(tools)

sys.path

['/Users/loveen/Desktop/Masters project/rl-multi-agent-traffic-nonexhaust-emissions-clean',
 '/Users/loveen/.pyenv/versions/3.9.10/lib/python39.zip',
 '/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9',
 '/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/lib-dynload',
 '',
 '/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages',
 '/opt/homebrew/opt/sumo/share/sumo/tools',
 '/opt/homebrew/opt/sumo/share/sumo/tools']

In [17]:
os.environ['SUMO_TOOLS'] = '/opt/homebrew/opt/sumo/share/sumo/tools'

In [18]:
print("SUMO_HOME is set to:", os.environ['SUMO_HOME'])

SUMO_HOME is set to: /opt/homebrew/opt/sumo/bin


In [19]:
# Verify the tools directory exists
# Verify the tools directory exists
os.path.exists(tools)

True

In [20]:
1/required_vehicle_rate

2.4390243902439024

In [22]:
seed = 285

In [24]:
sumo_tools = os.getenv('SUMO_TOOLS')

random_trips_cmd = f"python {sumo_tools}/randomTrips.py -n data/4x4grid_similar_to_resco_train_new_files/grid4x4.net.xml -o data/4x4grid_similar_to_resco_train_new_files/random_trips_{seed}.trips.xml -e 10000 --seed={seed} --period=2.4390243902439024 --validate"

# Use the %shell magic to execute the command
!{random_trips_cmd}

Success.
Success.


In [27]:
# Run duarouter to convert trips to routes with the specified output path
dua_router_cmd = f"duarouter -b 0 -e 5000 --route-files=data/4x4grid_similar_to_resco_train_new_files/random_trips_{seed}.trips.xml --net-file=data/4x4grid_similar_to_resco_train_new_files/grid4x4.net.xml --output-file=data/4x4grid_similar_to_resco_train_new_files/random_routes_seed_{seed}.rou.xml" # --vtype-output "data/cologne8/vtype.xml"
!{dua_router_cmd}

Success.up to time step: 5000.00  (5000.00/5000.00 = 100.00% done)       


In [59]:
# !python $SUMO_TOOLS/randomTrips.py -n "data/4x4grid_similar_to_resco_train_new_files/grid4x4.net.xml" -o "data/4x4grid_similar_to_resco_train_new_files/random_trips_{seed}.trips.xml" -e 100000 --seed={seed} --period=2.4390243902439024 --validate

In [32]:
# Run duarouter to convert trips to routes with the specified output path
# !duarouter -b 0 -e 5000 --route-files="data/4x4grid_similar_to_resco_train_new_files/random_trips_42.trips.xml" --net-file="data/4x4grid_similar_to_resco_train_new_files/grid4x4.net.xml" --output-file="data/4x4grid_similar_to_resco_train_new_files/random_routes_seed_42_testing.rou.xml" # --vtype-output "data/cologne8/vtype.xml" 

Loading net ... done.
Skipped until: 0.00
Routes found between time steps 0.00 and 5000.00.0 = 100.00% done)       
DijkstraRouter answered 1961 queries and explored 34.79 edges on average.
DijkstraRouter spent 0.00s answering queries (0.00ms on average).
Success.


In [219]:
# Load the routes file
original_cologne8 = ET.parse("/Users/loveen/Desktop/Masters project/rl-multi-agent-traffic-nonexhaust-emissions-clean/data/cologne8_resco_eval/cologne8.rou.xml")
root = original_cologne8.getroot()

original_cologne8_flows = {}
# Iterate over routes and convert them to flows
flow_id = 0
for trip in root.findall('trip'):
    # Create a flow element
    from_edge = trip.get('from')
    to_edge = trip.get('to')

    if (from_edge, to_edge) not in original_cologne8_flows.keys():
        original_cologne8_flows[(from_edge, to_edge)] = 0
    original_cologne8_flows[(from_edge, to_edge)] += 1

In [220]:
print("no of unique origin-destin pairs", len(original_cologne8_flows.keys()))

no of unique origin-destin pairs 579


In [221]:
print("total no of trips", sum(original_cologne8_flows.values()))

total no of trips 2046


In [222]:
(2046, 3600)
3600/2046, 2046/3600

resco_sec = 3600 
resco_no_vehicles = 2046
resco_tps = 2046/3600

resco_tps

0.5683333333333334

In [223]:
my_sec = 10000
my_no_vehicles = resco_tps*my_sec
requied_tps = 2046/3600

In [224]:
my_no_vehicles

5683.333333333334

In [225]:
# Load the routes file
my_curated_one = ET.parse("/Users/loveen/Desktop/Masters project/rl-multi-agent-traffic-nonexhaust-emissions-clean/data/cologne8/random_routes.rou.xml")
root = my_curated_one.getroot()

my_curated_one_flows = {}
# Iterate over routes and convert them to flows
flow_id = 0
for trip in root.findall('trip'):
    # Create a flow element
    from_edge = trip.get('from')
    to_edge = trip.get('to')

    if (from_edge, to_edge) not in my_curated_one_flows.keys():
        my_curated_one_flows[(from_edge, to_edge)] = 0
    my_curated_one_flows[(from_edge, to_edge)] += 1

In [226]:
print("no of unique origin-destin pairs", len(my_curated_one_flows.keys()))

no of unique origin-destin pairs 4595


In [227]:
print("total no of trips", sum(my_curated_one_flows.values()))

total no of trips 5475
